In [ ]:
import re
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

def preprocess_law_text(raw_text):
    """
    법령 텍스트에서 메뉴 항목과 부칙/별표를 제거하고 본문만 추출합니다.
    """
    # [시작점] "한눈보기" 이후 내용만 추출
    start_keyword = "한눈보기"
    if start_keyword in raw_text:
        content = raw_text.split(start_keyword)[-1].strip()
    else:
        content = raw_text

    # [종료점] "부칙" 혹은 "[별표" 이전까지만 남김
    end_patterns = [r"부\s*칙", r"\[별표"]
    for pattern in end_patterns:
        content = re.split(pattern, content)[0]
    
    return content.strip()

def get_chunks_from_file(file_path):
    # --- Step 1: 로컬 TXT 파일 로드 ---
    loader = TextLoader(file_path, encoding='utf-8')
    raw_documents = loader.load()
    
    # --- Step 2: 본문만 추출 (Cleaning) ---
    cleaned_text = preprocess_law_text(raw_documents[0].page_content)
    
    # 정제된 텍스트를 담은 새로운 Document 객체 생성
    cleaned_doc = Document(
        page_content=cleaned_text, 
        metadata={"source": file_path}
    )
    
    # --- Step 3: 의미 단위 분할 (Chunking) ---
    # 법령의 '제1조' 단위를 보존하기 위해 separators 설정
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,          # 원하는 청크 크기
        chunk_overlap=50,        # 앞뒤 문맥 겹침 정도
        separators=["\n제", "\n", " ", ""] 
    )
    
    chunks = text_splitter.split_documents([cleaned_doc])
    
    return chunks

file_path = "..\B-TEAM\사회복지_법령_전체.txt" 
final_chunks = get_chunks_from_file(file_path)

# # --- 실행 및 결과 확인 ---
# # 확인을 위한 출력
# print(f"✅ 총 {len(final_chunks)}개의 청크가 생성되었습니다.")
# print("-" * 30)
# print("첫 번째 청크 내용 예시:")
# print(final_chunks[0].page_content)

✅ 총 42개의 청크가 생성되었습니다.
------------------------------
첫 번째 청크 내용 예시:
성평등가족부 직제 시행규칙
[시행 2026. 1. 6.] [성평등가족부령 제4호, 2026. 1. 6., 일부개정]
성평등가족부(혁신행정법무담당관), 02-2100-6082
        제1장 총칙
제1조(목적) 이 규칙은 성평등가족부에 두는 보조기관ㆍ보좌기관의 직급 및 직급별 정원, 「정부조직법」 제2조제3항 및 제5항에 따라 실장ㆍ국장 밑에 두는 보조기관과 이에 상당하는 보좌기관의 설치 및 사무분장 등 「성평등가족부 직제」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.
        제2장 성평등가족부


<>:57: SyntaxWarning: invalid escape sequence '\B'
<>:57: SyntaxWarning: invalid escape sequence '\B'
C:\Users\USER\AppData\Local\Temp\ipykernel_28420\1970086736.py:57: SyntaxWarning: invalid escape sequence '\B'
  file_path = "..\B-TEAM\사회복지_법령_전체.txt" # 실제 파일 경로로 수정하세요
